In [ ]:
import numpy as np
import imutils
import cv2
import io
import os
import random
import shutil
import matplotlib.pyplot as plt
import skimage 
from skimage import filters 
from skimage.feature import greycomatrix, greycoprops
from skimage import data

In [ ]:
# function for image processing with segmentation
def image_processing(image):
                #Gaussian Blur
    blurGaussian = cv2.GaussianBlur(image,(5,5),5)
                #mean blur
    kernel = np.ones((5,5),np.float32)/25
    meanblur = cv2.filter2D(blurGaussian,-1,kernel)
                #Median Blur
    median = cv2.medianBlur(meanblur,5)
          #segmentation of images  using threshhold segmentation
    th =155
    max_value=255
    ret, out1 = cv2.threshold(median,th,max_value,cv2.THRESH_BINARY)
        #contours
    
    plt.subplot(1, 2, 2)
    plt.imshow(out1)
    plt.tick_params(axis='both', which='both',top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
    plt.title('Cropped Image')
    plt.show()
                    #segmenatation of images using historigram
    histimg=plt.hist(median.ravel(),256,[0,256])
    plt.show()

    iar = np.asarray(median)
    print(iar)
    iar.max()
    # 'contrast',‘dissimilarity’, ‘homogeneity’, ‘energy’, ‘correlation’, ‘ASM’
    glcm=g = greycomatrix(iar, [1,2], [0, np.pi/2], levels=260,normed=True, symmetric=True)

    contrast = greycoprops(glcm, 'contrast')
    print("the contrast of the images is ",contrast)
    dissimilarity = greycoprops(glcm, 'dissimilarity')
    print("the dissimilarity of the images is ",dissimilarity)
    homogeneity = greycoprops(glcm, 'homogeneity')
    print("the homogeneity of the images is ",homogeneity)
    energy = greycoprops(glcm, 'energy')
    print("the energy of the images is ",energy)
    correlation = greycoprops(glcm, 'correlation')
    print("the correlation of the images is ",energy)
    ASM = greycoprops(glcm, 'ASM')
    print("the ASM of the images is ",ASM)


    plt.plot(contrast,label="contrast")
    plt.plot(dissimilarity,label="dissimilarity")
    plt.plot(homogeneity,label="homogeneity")
    plt.plot(energy,label="energy")
    plt.plot(correlation,label="correlation")
    plt.plot(ASM,label="ASM")
    plt.title("GSLM MATRIX")
    plt.xlabel("OFFSITE")
    plt.ylabel("FEATURES")
    plt.legend()
    plt.show()
    return out1



In [ ]:
size_img=int(input("enter the re-sized images"))
IMG_DIR="G:\\brainMydataSet"
CATEGORIES = ["no","yes"]
X=[]
Y=[]
def create_data_set():
    for categories in CATEGORIES:
        path = os.path.join(IMG_DIR,categories)
        #this function is used to cencatenate the path with sub folder
        #class_num can classified categories and marked it as 0 and 1 form.
        class_num = CATEGORIES.index(categories) 
        for img in os.listdir(path):
             # load the image\n",
            img_array =cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
            resized_img = cv2.resize(img_array,(size_img,size_img),interpolation=cv2.INTER_CUBIC)
            processed_img=image_processing(resized_img)
            # normalize values
            normal_img = processed_img/255
            # convert image to numpy array and append it to X
            X.append(normal_img)
            # append a value of 1 to the target array if the image is in the folder named 'yes', otherwise append 0.
            #from classnum
            Y.append(class_num)
    x=np.array(X)
    y=np.array(Y)
    
    print(f'Number of examples is: {len(x)}')
    print(f'X shape is: {x.shape}')
    print(f'y shape is: {y.shape}')
    
    return x,y

In [ ]:
create_data_set()
print(np.ndim(X))


In [ ]:
# Shuffle two lists with same order 
# Using zip() + * operator + shuffle() 
temp = list(zip(X,Y)) 
random.shuffle(temp) 
X,y= zip(*temp) 


In [ ]:
y
np.shape(X)
print(np.ndim(X))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def split_data(X, y, test_size=0.2):
       
    """
    Splits data into training, development and test sets.
    Arguments:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    Returns:
        X_train: A numpy array with shape = (#_train_examples, image_width, image_height, #_channels)
        y_train: A numpy array with shape = (#_train_examples, 1)
        X_val: A numpy array with shape = (#_val_examples, image_width, image_height, #_channels)
        y_val: A numpy array with shape = (#_val_examples, 1)
        X_test: A numpy array with shape = (#_test_examples, image_width, image_height, #_channels)
        y_test: A numpy array with shape = (#_test_examples, 1)
    """
    
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, Y, test_size=0.3)

In [ ]:
print ("number of training examples = " + str(np.shape(X_train)[0]))
print ("number of development examples = " + str(np.shape(X_val)[0]))
print ("number of test examples = " + str(np.shape(X_test)[0]))
print ("X_train shape: " + str(np.shape(X_train)))
print ("Y_train shape: " + str(np.shape(y_train)))
print ("X_val (dev) shape: " + str(np.shape(X_val)))
print ("Y_val (dev) shape: " + str(np.shape(y_val)))
print ("X_test shape: " + str(np.shape(X_test)))
print ("Y_test shape: " + str(np.shape(y_test)))

In [ ]:
print(np.ndim(X_train))
print(np.ndim(X_test))
print(y_train)

In [ ]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return format"{h}:{m}:{round(s,1)}"

In [ ]:
def compute_f1_score(y_true, prob):
    # convert the vector of probabilities to a target vector
    y_pred = np.where(prob > 0.5, 1, 0)
    
    score = f1_score(y_true, y_pred)
    
    return score

# Let's build a convolutional neural network model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
def build_model(input_shape):
    """
    Arugments:
        input_shape: A tuple representing the shape of the input of the model. shape=(image_width, image_height, #_channels)
    Returns:
        model: A Model object.
    """
    # Define the input placeholder as a tensor with shape input_shape. 
    X_input = tf.keras.Input(input_shape)# shape=(?, 128, 128, 1)
    print(X_input.shape)
    
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((2, 2))(X_input) # shape=(?, 244, 244, 3)
    print(X.shape)
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) # shape=(?, 238, 238, 32)
    print(X.shape)
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool0')(X) # shape=(?, 59, 59, 32) 
    print(X.shape)
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool1')(X) # shape=(?, 14, 14, 32)
    print(X.shape)
    # FLATTEN X 
    X = Flatten()(X)# shape=(?, 6272)
    print(X.shape)
    # FULLYCONNECTED
    X = Dense(1, activation='sigmoid', name='fc')(X) # shape=(?, 1)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='BrainDetectionModel')
    
    return model

In [ ]:
input_shape = (size_img, size_img,1)

In [ ]:
model = build_model(input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
print(np.shape(X_train))
print(np.ndim(X_train))

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val= np.asarray(y_val)

In [ ]:
X_train = np.reshape(X_train,(177,size_img, size_img,1))
X_test = np.reshape(X_test,(38,size_img, size_img,1))
X_val = np.reshape(X_val,(38,size_img, size_img,1))

In [ ]:
print("Fit model on training data")
modelhistory = model.fit(x=X_train, y=y_train, batch_size=32, epochs=7, validation_data=(X_val, y_val))

In [ ]:
modelhistory.history

In [ ]:
# plot model performance
acc = modelhistory.history['accuracy']
val_acc = modelhistory.history['val_accuracy']
loss = modelhistory.history['loss']
val_loss = modelhistory.history['val_loss']
epochs_range = range(1, len(modelhistory.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

In [ ]:
#To save model ..
model.save('BrainTumorDetection.model')

In [ ]:
filepath1="G:\\brainMydataSet\\yes\\Y104.jpg"
filepath2="G:\\brainMydataSet\\no\\25 no.jpg"

In [ ]:

import cv2
import tensorflow as tf

CATEGORIES = ["no","yes"]


def prepare(filepath):
    IMG_SIZE = 128  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("BrainTumorDetection.model")

prediction = model.predict([prepare(filepath1)])
print(prediction)
# will be a list in a list.
cate=(int(str(float(prediction)).split('.')[0]))
print(cate)
print(CATEGORIES[cate])

In [ ]:
prediction = model.predict([prepare(filepath2)])  # will be a list in a list.
print(prediction[0][0])
type(prediction)
np.ndim(prediction)

# Svm model

In [ ]:
#Svm model
from sklearn import svm

classifier = svm.SVC(kernel='linear')
classifier.fit(X[:176], y[:176])
predictions = classifier.predict(X[176:])

In [ ]:
#Import svm model
from sklearn import svm


In [ ]:
clf = svm.SVC(kernel='linear')
# Linear Kernel
#Train the model using the training sets
print(clf.fit(d2_train_dataset, y_train))
#Predict the response for test dataset
y_pred=clf.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
X_train = np.reshape(X_train,(177,size_img, size_img,1))
X_test = np.reshape(X_test,(177,size_img, size_img,1))

In [ ]:
nsamples, nx, ny = np.shape(X_train)
ntsamples, ntx, nty = np.shape(X_test)
X_train = np.reshape(X_train,(nsamples,nx*ny))
X_test = np.reshape(X_test,(ntsamples,ntx*nty))
print(np.shape(X_train))
print(np.shape(X_test))

In [1]:
import cv2
import numpy as np
import os
import glob
import mahotas as mt
from sklearn.svm import LinearSVC

IMG_DIR="G:\\brainMydataSet"
CATEGORIES = ["no","yes"]

for categories in CATEGORIES:
    path = os.path.join(IMG_DIR,categories) #this function is used to cencatenate the path with sub folder
    for img in os.listdir(path):
        img_array =cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        break
    break
        # calculate haralick texture features for 4 types of adjacency
textures = mt.features.haralick(img_array)
print(textures)
        # take the mean of it and return it
ht_mean = textures.mean(axis=0)
print(ht_mean)

[[ 6.38458940e-02  3.90365912e+01  9.85222151e-01  1.32078054e+03
   5.56855287e-01  5.95298862e+01  5.24408557e+03  6.20515599e+00
   8.20527642e+00  9.48254082e-04  2.82972083e+00 -4.82824630e-01
   9.97299387e-01]
 [ 6.04977715e-02  6.61910773e+01  9.74955505e-01  1.32146960e+03
   4.98364775e-01  5.96245283e+01  5.21968733e+03  6.22110494e+00
   8.53593482e+00  7.71744015e-04  3.18530157e+00 -4.23288702e-01
   9.94875630e-01]
 [ 6.38321570e-02  3.03696873e+01  9.88503116e-01  1.32077907e+03
   5.53532560e-01  5.95299367e+01  5.25274657e+03  6.20343372e+00
   8.19894006e+00  9.39697872e-04  2.81247410e+00 -4.84013625e-01
   9.97334103e-01]
 [ 6.04988698e-02  6.58855680e+01  9.75071100e-01  1.32146960e+03
   4.98530070e-01  5.96245283e+01  5.21999284e+03  6.22212037e+00
   8.53706380e+00  7.71363299e-04  3.19191215e+00 -4.23080163e-01
   9.94864016e-01]]
[ 6.21686731e-02  5.03707309e+01  9.80937968e-01  1.32112470e+03
  5.26820673e-01  5.95772199e+01  5.23412808e+03  6.21295376e+00
 

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(2, 3, activation='relu', input_shape=tf.keras.Input(input_shape)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))


In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val= np.asarray(y_val)